In [3]:
#!pip install seaborn
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json
import seaborn as sns


In [20]:
data = pd.read_csv('../daou/danawa_crawling_result_강사님.csv')
#data = pd.read_csv('../daou/danawa_crawling_result.csv')
data.head()
#data.info()

,상품명,스펙 목록,가격
0,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔...,458920
1,베이스어스 차량용 청소기 A3 (해외구매),"차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필...",48360
2,LG전자 오브제컬렉션 코드제로 A9S AU9982,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이...,1038980
3,삼성전자 비스포크 제트 VS20B956AX,핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 220W / [구성] 바닥 / 솔...,539990
4,샤오미 CLEANFLY 차량용 무선 청소기 4세대 H2 (해외구매),"차량용청소기 / 무선 / 흡입력: 16,800Pa / 최대출력: 120W / 헤파필...",63330


In [21]:
# 회사명 모델명 분리
company_list = []
product_list = []
for i in data.상품명:
    i_info = i.split(" ", 1)   # 1을 넣어야 앞에 한개 공백만 분리
    company = i_info[0]
    product = i_info[1]
    company_list.append(company)
    product_list.append(product)

#product_list
#company_list


In [35]:
# .. 스펙목록 분리(카테고리, 사용시간, 흡입력)
spec_list = data['스펙 목록'][0].split('/')

# 
category_list = []
use_time_list = []
suction_list = []

use_time = None
suction = None
category = None

for spec_data in data['스펙 목록']:

    # 스펙분리
    spec_list = spec_data.split(' / ')

    # 카테고리 추출
    category = spec_list[0]
    category_list.append(category)

    # 사용시간, 흡입력 추출
    for spec in spec_list:
        if '사용시간' in spec:
            use_time = spec.split(":")[1].strip()
        elif '흡입력' in spec:
            suction = spec.split(":")[1].strip()
    use_time_list.append(use_time)
    suction_list.append(suction)

In [40]:
print('카테고리', len(category_list), category_list[0:5])
print('사용시간', len(use_time_list), use_time_list[0:5])
print('흡입력', len(suction_list), suction_list[0:5])


# --> 시간, 흡입력 단위가 다름

카테고리 400 ['핸디스틱청소기', '차량용청소기', '핸디스틱청소기', '핸디스틱청소기', '차량용청소기']
사용시간 400 ['1시간', '45분', '1시간', '1시간', '30분']
흡입력 400 ['210W', '15,000Pa', '220W', '220W', '16,800Pa']


In [74]:
# 시간단위 통일
def convert_time_minute(time):
    try:
        if '시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
            else:
                minute = 0
        else:
            hour = 0
            minute = time.split('분')[0]
        return int(hour)*60 + int(minute)
    except:
        return None

In [ ]:
cvrt_use_time_list = [ convert_time_minute(i) for i in use_time_list ]
cvrt_use_time_list

In [82]:
# 흡입력 통이 W = 100Pa, W 지우기
def convert_suction_unit(value):
    try:
        value = value.upper()
        if 'AW' in value or 'W' in value:
            result = value.replace('A', '').replace('W','')
            result = int(result.replace(',',""))
        elif 'PA' in value:
            result = value.replace('PA',"")
            result = int(int(result.replace(',',''))/100)
        else:
            result = None
        return result    
    except:
        return None

In [ ]:
cvrt_suction_list = [ convert_suction_unit(i) for i in  suction_list ]
cvrt_suction_list

In [85]:
# 전처리 결과 엑셀로 저장
pd_data = pd.DataFrame()
pd_data['카테고리'] = category_list
pd_data['회사명'] = company_list
pd_data['제품'] = product_list
pd_data['가격'] = data['가격']
pd_data['사용시간'] = cvrt_use_time_list
pd_data['흡입력'] = cvrt_suction_list
pd_data

,카테고리,회사명,제품,가격,사용시간,흡입력
0,핸디스틱청소기,삼성전자,비스포크 제트 VS20A956A3,458920,60.0,210.0
1,차량용청소기,베이스어스,차량용 청소기 A3 (해외구매),48360,45.0,150.0
2,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AU9982,1038980,60.0,220.0
3,핸디스틱청소기,삼성전자,비스포크 제트 VS20B956AX,539990,60.0,220.0
4,차량용청소기,샤오미,CLEANFLY 차량용 무선 청소기 4세대 H2 (해외구매),63330,30.0,168.0
...,...,...,...,...,...,...
395,핸디스틱청소기,다이슨,V11 컴플리트,938360,60.0,185.0
396,핸디스틱청소기,DIBEA,D18,119960,45.0,90.0
397,핸디스틱청소기,캐치웰,CX11 PLUS,152150,60.0,200.0
398,차량용청소기,휴대용,청소기 트리플블랙 Z5,47710,15.0,50.0


In [86]:
pd_data.to_csv('../daou/다나와시각화입력자료', index=False)

In [87]:
pd_data['카테고리'].value_counts()

핸디스틱청소기             221
차량용청소기               89
물걸레청소기               31
핸디청소기                30
스틱청소기                 7
욕실청소기                 7
침구청소기                 5
핸디/스틱청소기+로봇청소기세트      3
업소용청소기                3
차량용선풍기                1
핸디/스틱청소기+공기청정기세트      1
수영장청소기                1
물걸레청소기+스팀청소기          1
Name: 카테고리, dtype: int64

카테고리만 뽑아서 분석
* 가격, 사용시간, 흡입력 평균
* 3가지 평균 이상 제품만 출력

In [94]:
pd_data_final = pd_data[pd_data['카테고리'].isin(['핸디스틱청소기'])]
pd_data_final

,카테고리,회사명,제품,가격,사용시간,흡입력
0,핸디스틱청소기,삼성전자,비스포크 제트 VS20A956A3,458920,60.0,210.0
2,핸디스틱청소기,LG전자,오브제컬렉션 코드제로 A9S AU9982,1038980,60.0,220.0
3,핸디스틱청소기,삼성전자,비스포크 제트 VS20B956AX,539990,60.0,220.0
5,핸디스틱청소기,일렉트로룩스,에르고라피도 ZB3512IG,160080,35.0,168.0
6,핸디스틱청소기,LG전자,코드제로 A9S AT9200BA,576860,60.0,200.0
...,...,...,...,...,...,...
392,핸디스틱청소기,샤오미,Lydsto H4,369000,23.0,250.0
395,핸디스틱청소기,다이슨,V11 컴플리트,938360,60.0,185.0
396,핸디스틱청소기,DIBEA,D18,119960,45.0,90.0
397,핸디스틱청소기,캐치웰,CX11 PLUS,152150,60.0,200.0


In [95]:
pd_data_final.to_csv('2_result_preprocessing.csv', index=False)